In [62]:
import data_processing as dclean
from config import app
from models import Hive
import pandas as pd
import numpy as np

In [63]:
with app.app_context():
    hives = [hive.to_dict() for hive in Hive.query.all()]

In [64]:
def aggregate_data(df_normalized):
    df = df_normalized.copy()
    df_filtered = df.loc[~df['date_pulled'].isna()].copy()

    df_filtered['count'] = 1

    # convert to datetime
    df_filtered['date_checked'] = pd.to_datetime(df_filtered['date_checked'])

    # Add season column based on month of inspection
    df_filtered['season'] = df_filtered['date_checked'].dt.month % 12 // 3 + 1  # 1=Winter, 2=Spring, 3=Summer, 4=Fall

    hive_columns = ['hive_id', 'date_added', 'material', 'city', 'state']
    honey_pull_columns = ['honey_pull_id', 'date_reset', 'date_pulled', 'weight']

    # # Aggregating by honey_pull_id
    df_aggregated = df_filtered.groupby(hive_columns+honey_pull_columns).agg({
        'count': 'sum',
        'temp': 'mean',
        'bias': 'mean',
        'num_pollen_patties': 'mean',
        'num_sugar_syrup_frames': 'mean',
        'varroa_mite_count': 'mean',
        'humidity': 'mean',
        'ants_present': 'sum',
        'slugs_present': 'sum',
        'hive_beetles_present': 'sum',
        'wax_moths_present': 'sum',
        'wasps_hornets_present': 'sum',
        'mice_present': 'sum',
        'robber_bees_present': 'sum',
        'has_chalkbrood': 'sum',
        'has_twisted_larvae': 'sum'
    }).reset_index()

    df_aggregated['days'] = (pd.to_datetime(df_aggregated['date_pulled']) - pd.to_datetime(df_aggregated['date_reset'])).dt.days

    return df_aggregated

In [65]:
hives_mod = dclean.rename_ids(hives)
print('renamed ids...')

df_normalized = dclean.normalize_data(hives_mod)
print('df normalized...')

df_aggregated = aggregate_data(df_normalized)
print('df aggregated...')

json_normalized = df_normalized.to_dict(orient='list')
print('json normalized...')

json_aggregated = df_aggregated.to_dict(orient='list')
print('json aggregated...')

renamed ids...
df normalized...
df aggregated...
json normalized...
json aggregated...


In [78]:
def binarize_data(data):
    categorical_columns = ['fate', 'activity_surrounding_hive', 'stability_in_hive']
    
    # One-hot encoding categorical columns, dropping the first category to avoid multicollinearity
    binarized_data = pd.get_dummies(data, columns=categorical_columns, drop_first=True)

    return binarized_data

In [79]:
binarize_data(df_normalized)

,varroa_mite_count,has_twisted_larvae,slugs_present,inspection_id,num_pollen_patties,robber_bees_present,formic_acid_dosage,apistan_dosage,super_count,weather_conditions,...,city,postal_code,address_line,fate_Split,fate_Swarmed,fate_Thriving,activity_surrounding_hive_Low,activity_surrounding_hive_Medium,stability_in_hive_Low,stability_in_hive_Medium
0,39.0,True,False,2755.0,1.0,True,2.0,0.0,3.0,Sunny,...,Lake Monicachester,05154,34258 Alan Extension,False,True,False,True,False,False,True
1,41.0,False,False,2756.0,0.0,False,0.0,0.0,3.0,Overcast,...,Lake Monicachester,05154,34258 Alan Extension,False,False,True,False,False,False,False
2,59.0,True,False,2757.0,0.0,False,0.0,0.0,3.0,Snowy,...,Lake Monicachester,05154,34258 Alan Extension,False,False,False,False,False,False,False
3,70.0,True,False,2758.0,0.0,False,1.0,0.0,3.0,Sunny,...,Lake Monicachester,05154,34258 Alan Extension,False,False,False,False,True,True,False
4,40.0,True,False,2759.0,2.0,False,0.0,0.0,3.0,Windy,...,Lake Monicachester,05154,34258 Alan Extension,False,False,False,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1062,60.0,True,False,3817.0,1.0,False,1.0,0.0,1.0,Overcast,...,West Jonathanberg,07036,90686 Flores Inlet,False,False,False,True,False,True,False
1063,59.0,True,False,3818.0,2.0,True,0.0,0.0,1.0,Rainy,...,West Jonathanberg,07036,90686 Flores Inlet,False,False,True,False,False,False,True
1064,33.0,False,False,3819.0,2.0,False,2.0,1.0,1.0,Sunny,...,West Jonathanberg,07036,90686 Flores Inlet,False,True,False,False,True,False,False
1065,28.0,True,False,3820.0,2.0,False,2.0,0.0,1.0,Sunny,...,West Jonathanberg,07036,90686 Flores Inlet,False,False,False,True,False,True,False


In [47]:
list(json_aggregated.keys())

['hive_id',
 'date_added',
 'material',
 'city',
 'state',
 'honey_pull_id',
 'date_reset',
 'date_pulled',
 'weight',
 'count',
 'temp',
 'bias',
 'num_pollen_patties',
 'num_sugar_syrup_frames',
 'varroa_mite_count',
 'humidity',
 'ants_present',
 'slugs_present',
 'hive_beetles_present',
 'wax_moths_present',
 'wasps_hornets_present',
 'mice_present',
 'robber_bees_present',
 'has_chalkbrood',
 'has_twisted_larvae',
 'days']

In [6]:
df_normalized['slugs_present']

0       False
1       False
2       False
3       False
4       False
        ...  
1062    False
1063    False
1064    False
1065    False
1066    False
Name: slugs_present, Length: 1067, dtype: object

In [31]:
df_aggregated

,hive_id,date_added,material,city,state,honey_pull_id,date_reset,date_pulled,weight,count,temp,bias,num_pollen_patties,num_sugar_syrup_frames,varroa_mite_count,humidity,ants_present,hive_beetles_present,has_chalkbrood
0,80,2024-05-25,Other,Lake Monicachester,Nevada,162.0,2024-05-25,2024-09-23,60.222476,18,22.944444,1.277778,0.777778,0.944444,42.277778,64.816667,13,9,13
1,80,2024-05-25,Other,Lake Monicachester,Nevada,163.0,2024-09-28,2025-01-02,165.499312,14,24.914286,1.5,0.642857,1.0,26.0,68.207143,5,7,6
2,81,2024-05-25,Polystyrene,Malonebury,Mississippi,165.0,2024-05-25,2024-11-05,6.089138,24,22.633333,1.208333,1.166667,1.041667,83.666667,60.479167,22,19,12
3,82,2022-03-01,Other,Haydenfort,South Carolina,167.0,2022-03-01,2022-08-06,106.919496,23,23.186957,1.521739,0.869565,1.043478,30.130435,68.034783,12,10,13
4,82,2022-03-01,Other,Haydenfort,South Carolina,168.0,2022-08-11,2023-01-24,29.439186,24,22.645833,2.083333,0.833333,1.041667,89.5,67.291667,24,22,16
5,82,2022-03-01,Other,Haydenfort,South Carolina,169.0,2023-02-03,2023-08-02,172.299908,26,22.057692,1.038462,1.461538,1.0,26.807692,59.573077,10,7,14
6,82,2022-03-01,Other,Haydenfort,South Carolina,170.0,2023-08-07,2023-11-26,13.170006,16,23.225,1.375,0.8125,1.25,93.875,60.1875,16,13,3
7,82,2022-03-01,Other,Haydenfort,South Carolina,171.0,2023-12-07,2024-06-04,5.197530,26,22.873077,1.653846,0.884615,1.076923,94.884615,59.611538,25,25,14
8,82,2022-03-01,Other,Haydenfort,South Carolina,172.0,2024-06-17,2024-10-22,47.743505,19,26.173684,1.842105,0.947368,0.894737,49.842105,69.105263,18,10,11
9,83,2023-02-12,Polystyrene,Richardmouth,Connecticut,174.0,2023-02-12,2023-05-15,143.043994,14,23.214286,1.642857,1.071429,0.857143,24.285714,61.071429,8,1,6


In [19]:
graph_data = dclean.process_data_for_graphing(hives)

In [22]:
graph_data

{'normalized': {'has_twisted_larvae': [True,
   False,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   False,
   True,
   False,
   False,
   True,
   False,
   False,
   False,
   True,
   False,
   True,
   True,
   True,
   True,
   True,
   False,
   True,
   True,
   False,
   False,
   False,
   True,
   True,
   False,
   False,
   False,
   False,
   True,
   True,
   True,
   True,
   False,
   False,
   True,
   False,
   True,
   True,
   True,
   False,
   False,
   True,
   True,
   True,
   False,
   True,
   True,
   False,
   False,
   True,
   False,
   True,
   False,
   True,
   False,
   True,
   False,
   True,
   True,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   True,
   False,
   True,
   True,
   False,
   True,
   True,
   True,
   False,
   False,
   True,
   False,
   False,
   False,
   True,
   False,
   False,
   True,
   False,
   True,
   False,
   True,
   False,
   True,
   Fa

In [20]:
list(graph_data['aggregated'].keys())

['hive_id',
 'date_added',
 'material',
 'city',
 'state',
 'honey_pull_id',
 'date_reset',
 'date_pulled',
 'weight',
 'count',
 'temp',
 'bias',
 'num_pollen_patties',
 'num_sugar_syrup_frames',
 'varroa_mite_count',
 'humidity',
 'ants_present',
 'hive_beetles_present',
 'has_chalkbrood']

In [13]:

df_subset = df_aggregated.head(10)

df_subset.to_dict()

{'date_added': {0: '2022-10-12',
  1: '2022-10-12',
  2: '2022-10-12',
  3: '2022-10-12',
  4: '2022-10-12',
  5: '2022-10-12',
  6: '2023-02-01',
  7: '2023-02-01',
  8: '2023-02-01',
  9: '2023-02-01'},
 'material': {0: 'Polystyrene',
  1: 'Polystyrene',
  2: 'Polystyrene',
  3: 'Polystyrene',
  4: 'Polystyrene',
  5: 'Polystyrene',
  6: 'Polystyrene',
  7: 'Polystyrene',
  8: 'Polystyrene',
  9: 'Polystyrene'},
 'city': {0: 'Amandastad',
  1: 'Amandastad',
  2: 'Amandastad',
  3: 'Amandastad',
  4: 'Amandastad',
  5: 'Amandastad',
  6: 'Sanchezview',
  7: 'Sanchezview',
  8: 'Sanchezview',
  9: 'Sanchezview'},
 'state': {0: 'Michigan',
  1: 'Michigan',
  2: 'Michigan',
  3: 'Michigan',
  4: 'Michigan',
  5: 'Michigan',
  6: 'Michigan',
  7: 'Michigan',
  8: 'Michigan',
  9: 'Michigan'},
 'honey_pull_id': {0: 139,
  1: 140,
  2: 141,
  3: 142,
  4: 143,
  5: 144,
  6: 146,
  7: 147,
  8: 148,
  9: 149},
 'date_reset': {0: '2022-10-12',
  1: '2023-01-30',
  2: '2023-05-31',
  3: '2023

In [18]:
json_normalized = df_normalized.head().to_dict()

In [19]:
json_aggregated = df_aggregated.head().to_dict()

In [20]:
graph_data = {'normalized': json_normalized, 'aggregated': json_aggregated}

In [21]:
graph_data

{'normalized': {'humidity': {0: 86.0, 1: 85.0, 2: 58.7, 3: 50.7, 4: 87.8},
  'varroa_mite_count': {0: 31.0, 1: 58.0, 2: 60.0, 3: 17.0, 4: 91.0},
  'bias': {0: 2, 1: 2, 2: 2, 3: 3, 4: 2},
  'inspection_id': {0: 2121, 1: 2122, 2: 2123, 3: 2124, 4: 2125},
  'has_chalkbrood': {0: False, 1: True, 2: True, 3: True, 4: False},
  'ants_present': {0: True, 1: False, 2: True, 3: False, 4: True},
  'apistan_dosage': {0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0},
  'stability_in_hive': {0: 'High', 1: 'Low', 2: 'Low', 3: 'High', 4: 'Medium'},
  'num_pollen_patties': {0: 2, 1: 0, 2: 2, 3: 0, 4: 0},
  'mice_present': {0: True, 1: True, 2: False, 3: True, 4: False},
  'honey_pull_id': {0: 122, 1: 122, 2: 122, 3: 122, 4: 122},
  'robber_bees_present': {0: True, 1: False, 2: True, 3: False, 4: False},
  'hive_beetles_present': {0: True, 1: True, 2: True, 3: False, 4: False},
  'wasps_hornets_present': {0: True, 1: False, 2: True, 3: False, 4: False},
  'num_sugar_syrup_frames': {0: 1, 1: 2, 2: 2, 3: 1, 4: 1}

In [3]:
death_count = df_normalized[['fate', 'date_checked']].loc[df_normalized['fate']=='Dead']
len(death_count)

240